# Tutorial: Scraping web pages using requests and scrapy

In this tutorial, we will be using the `scrapy` library, specifically scrapy's XPath based `Selector` objects, to extract the required data from a particular webpage.

1. HTML (Hypertext Markup Language): HTML is the standard markup language for documents designed to be displayed in a web browser. When we want to scrape the content from a particular website, we are often given a html file.


2. XPath: XPath is an expression language designed to support the query or transformation of XML documents. This works with HTML documents too. 

In [1]:
from scrapy import Selector
import requests

import pandas as pd
import re
from typing import List

## Case study 1. Scraping tables from a wikipedia page

In [2]:
url1 = 'https://en.wikipedia.org/wiki/World_population'
response = requests.get(url1)
response.status_code

200

In [3]:
# get the HTML file as a string
html_content = response.content
html_content[:30]

b'<!DOCTYPE html>\n<html class="c'

In [4]:
# create a selector object
full_sel = Selector(text=html_content)
full_sel

<Selector query=None data='<html class="client-nojs vector-featu...'>

The `.xpath` method for a selector object can be used to query the elements within the `Selector`. It outputs a  `SelectorList` object - list of `Selector` objects that match the query.  


`SelectorList` extends a regular list with a few additional methods for `xpath` querying and extracting information.

In [5]:
# select all tables in page -> returns a SelectorList object
tables = full_sel.xpath('//table')

print(f'Type: {type(tables)}')
print(f'Number of tables: {len(tables)}')
print('\nSelectors for first four tables:')
for table in tables[:4]:
    print(table)

Type: <class 'scrapy.selector.unified.SelectorList'>
Number of tables: 28

Selectors for first four tables:
<Selector query='//table' data='<table class="wikitable" style="text-...'>
<Selector query='//table' data='<table class="box-Notice plainlinks m...'>
<Selector query='//table' data='<table class="wikitable sortable">\n<c...'>
<Selector query='//table' data='<table class="wikitable sortable">\n\n<...'>


In [6]:
# Attributes of first 4 tables
for i,table in enumerate(tables[:4]):
    print(f'Attribute for Table {i+1}: \n{table.attrib}')

Attribute for Table 1: 
{'class': 'wikitable', 'style': 'text-align:center; float:right; clear:right; margin-left:8px; margin-right:0;'}
Attribute for Table 2: 
{'class': 'box-Notice plainlinks metadata ambox ambox-notice', 'role': 'presentation', 'style': 'width:100%;margin:0;'}
Attribute for Table 3: 
{'class': 'wikitable sortable'}
Attribute for Table 4: 
{'class': 'wikitable sortable'}


Subset only tables with class "wikitable". (Exact match)

In [7]:
wikitables_exact = full_sel.xpath('//table[@class="wikitable"]')

print(f'Number of tables: {len(wikitables_exact)}')

print('\nFirst four tables:')
for table in wikitables_exact[:4]:
    print(table)

Number of tables: 7

First four tables:
<Selector query='//table[@class="wikitable"]' data='<table class="wikitable" style="text-...'>
<Selector query='//table[@class="wikitable"]' data='<table class="wikitable" style="font-...'>
<Selector query='//table[@class="wikitable"]' data='<table class="wikitable" style="text-...'>
<Selector query='//table[@class="wikitable"]' data='<table class="wikitable" style="text-...'>


Subset all tables whose class attribute contains "wikitable"

In [8]:
wikitables = full_sel.xpath('//table[contains(@class,"wikitable")]')
print(f'Number of tables: {len(wikitables)}')

print('\nFirst four tables:')
for table in wikitables[:4]:
    print(table)

Number of tables: 14

First four tables:
<Selector query='//table[contains(@class,"wikitable")]' data='<table class="wikitable" style="text-...'>
<Selector query='//table[contains(@class,"wikitable")]' data='<table class="wikitable sortable">\n<c...'>
<Selector query='//table[contains(@class,"wikitable")]' data='<table class="wikitable sortable">\n\n<...'>
<Selector query='//table[contains(@class,"wikitable")]' data='<table class="wikitable" style="font-...'>


### Extracting data from a table

Let's looks the second table in the list `wikitables`. This table containts the 2020 estimates of the population by region.

The `extract` method returns the corresponding HTML code as a string

In [9]:
print(wikitables[1].extract())

<table class="wikitable sortable">
<caption>Population by region (2020 estimates)
</caption>
<tbody><tr>
<th scope="col">Region
</th>
<th scope="col">Density<br><small>(inhabitants/km<sup>2</sup>)</small>
</th>
<th scope="col">Population<br><small>(millions)</small>
</th>
<th scope="col">Most populous country
</th>
<th scope="col">Most populous city (metropolitan area)
</th></tr>
<tr>
<td>Asia
</td>
<td style="text-align:right">104.1
</td>
<td style="text-align:right">4,641
</td>
<td data-sort-value="14118e5">1,418,459,382 – <b><span class="flagicon"><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/23px-Flag_of_India.svg.png" decoding="async" width="23" height="15" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/35px-Flag_of_India.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/45px-Flag_of_India.svg.png 2x" data-file-width="1350" data-file-height="900"> </span><a href="/wiki/In

In [11]:
# get the caption
wikitables[1].xpath('./caption/text()').get()

'Population by region (2020 estimates)\n'

In [12]:
# get the rows
rows = wikitables[1].xpath('./tbody//tr')
print(f'Number of rows (including header): {len(rows)}')

Number of rows (including header): 8


In [14]:
def parse_row(row:Selector) -> List[str]:
    '''
    Parses a html row into a list of individual elements
    '''
    cells = row.xpath('.//th | .//td')
    row_data = []
    
    for cell in cells:
        cell_text = cell.xpath('normalize-space(.)').get()
        cell_text = re.sub(r'<.*?>', ' ', cell_text)  # Remove remaining HTML tags
        # if there are br tags, there will be some binary characters
        cell_text = cell_text.replace('\xa0', '')  # Remove \xa0 characters
        row_data.append(cell_text)
    
    return row_data

In [15]:
header = parse_row(rows[0])
header

['Region',
 'Density(inhabitants/km2)',
 'Population(millions)',
 'Most populous country',
 'Most populous city (metropolitan area)']

In [16]:
table_data = [parse_row(row) for row in rows[1:]]
table_data

[['Asia',
  '104.1',
  '4,641',
  '1,418,459,382 – India',
  '13,515,000 – Tokyo Metropolis(37,400,000 – Greater Tokyo Area)'],
 ['Africa',
  '44.4',
  '1,340',
  '0,211,401,000 – Nigeria',
  '09,500,000 – Cairo(20,076,000 – Greater Cairo)'],
 ['Europe',
  '73.4',
  '747',
  '0,146,171,000 – Russia, approx. 110million in Europe',
  '13,200,000 – Moscow(20,004,000 – Moscow metropolitan area)'],
 ['Latin America',
  '24.1',
  '653',
  '0,214,103,000 – Brazil',
  '12,252,000 – São Paulo City(21,650,000 – São Paulo Metro Area)'],
 ['Northern America[note 1]',
  '14.9',
  '368',
  '0,332,909,000 – United States',
  '08,804,000 – New York City(23,582,649 – New York metropolitan area[90])'],
 ['Oceania', '5', '42', '0,025,917,000 – Australia', '05,367,000 – Sydney'],
 ['Antarctica',
  '~0',
  '0.004[89]',
  'N/A[note 2]',
  '00,001,258 – McMurdo Station']]

In [17]:
# convert table into a data frame
pop_by_region_df = pd.DataFrame(table_data,columns=header)
pop_by_region_df

,Region,Density(inhabitants/km2),Population(millions),Most populous country,Most populous city (metropolitan area)
0,Asia,104.1,"4,641","1,418,459,382 – India","13,515,000 – Tokyo Metropolis(37,400,000 – Gre..."
1,Africa,44.4,"1,340","0,211,401,000 – Nigeria","09,500,000 – Cairo(20,076,000 – Greater Cairo)"
2,Europe,73.4,747,"0,146,171,000 – Russia, approx. 110million in ...","13,200,000 – Moscow(20,004,000 – Moscow metrop..."
3,Latin America,24.1,653,"0,214,103,000 – Brazil","12,252,000 – São Paulo City(21,650,000 – São P..."
4,Northern America[note 1],14.9,368,"0,332,909,000 – United States","08,804,000 – New York City(23,582,649 – New Yo..."
5,Oceania,5,42,"0,025,917,000 – Australia","05,367,000 – Sydney"
6,Antarctica,~0,0.004[89],N/A[note 2],"00,001,258 – McMurdo Station"


The cell below contains a function to parse the entire table

In [18]:
def parse_table_as_df(table_sel:Selector,header:bool=True) -> pd.DataFrame:
    
    # extract rows
    rows = table_sel.xpath('./tbody//tr')
    
    # parse header and the remaining rows
    columns = None
    start_row = 0
    if header:
        columns = parse_row(rows[0])
        start_row += 1
        
    table_data = [parse_row(row) for row in rows[start_row:]]
    
    # return data frame
    return pd.DataFrame(table_data,columns=columns)

In [19]:
# pull the third wikitable
parse_table_as_df(wikitables[2])

,Rank,Country / Dependency,Population,Percentage of the world,Date,Source (official or from the United Nations)
0,1,India,"1,425,775,850",17.8%,14 Apr 2023,UN projection[91]
1,2,China,"1,412,600,000",17.6%,31 Dec 2021,National annual estimate[92]
2,3,United States,"334,753,588",4.17%,18 May 2023,National population clock[93]
3,4,Indonesia,"275,773,800",3.43%,1 Jul 2022,National annual estimate[94]
4,5,Pakistan,"229,488,994",2.86%,1 Jul 2022,UN projection[95]
5,6,Nigeria,"216,746,934",2.70%,1 Jul 2022,UN projection[95]
6,7,Brazil,"216,156,993",2.69%,18 May 2023,National population clock[96]
7,8,Bangladesh,"168,220,000",2.09%,1 Jul 2020,Annual Population Estimate[97]
8,9,Russia,"147,190,000",1.83%,1 Oct 2021,2021 preliminary census results[98]
9,10,Mexico,"128,271,248",1.60%,31 Mar 2022,


## Case-study 2: Extracting alcohol related mortality stats from CDC

In [20]:
url2 = 'https://www.cdc.gov/nchs/fastats/alcohol.htm'
response = requests.get(url2)

# Create a scrapy Selector from the response content
selector = Selector(text=response.content)

In [21]:
div_sel_list = selector.xpath('//div[@class="card mb-3"]')
div_sel_list

[<Selector query='//div[@class="card mb-3"]' data='<div class="card mb-3"><div class="ca...'>,
 <Selector query='//div[@class="card mb-3"]' data='<div class="card mb-3"><div class="ca...'>,
 <Selector query='//div[@class="card mb-3"]' data='<div class="card mb-3"><div class="ca...'>]

In [22]:
print(div_sel_list[1].extract())

<div class="card mb-3"><div class="card-header h4 bg-primary">Mortality</div><div class="card-body bg-white"><ul>
<li>Number of alcoholic liver disease deaths: 33,098</li>
<li>Alcoholic liver disease deaths per 100,000 population: 10.0</li>
</ul>
<p>Source: <a href="https://wonder.cdc.gov/controller/saved/D158/D321F155">National Vital Statistics System – Mortality Data (2021) via CDC WONDER</a></p>
<ul>
<li>Number of alcohol-induced deaths, excluding accidents and homicides: 54,258</li>
<li>Alcohol-induced deaths, excluding accidents and homicides per 100,000 population: 16.3</li>
</ul>
<p>Source: <a href="https://wonder.cdc.gov/controller/saved/D158/D321F156">National Vital Statistics System – Mortality Data (2021) via CDC WONDER </a></p>
</div></div>


In [23]:
div_sel = selector.xpath('//div[@class="card mb-3" and div[1][text()="Mortality"]]')
print(div_sel.extract())

['<div class="card mb-3"><div class="card-header h4 bg-primary">Mortality</div><div class="card-body bg-white"><ul>\n<li>Number of alcoholic liver disease deaths: 33,098</li>\n<li>Alcoholic liver disease deaths per 100,000 population: 10.0</li>\n</ul>\n<p>Source: <a href="https://wonder.cdc.gov/controller/saved/D158/D321F155">National Vital Statistics System – Mortality Data (2021) via CDC WONDER</a></p>\n<ul>\n<li>Number of alcohol-induced deaths, excluding accidents and homicides: 54,258</li>\n<li>Alcohol-induced deaths, excluding accidents and homicides per 100,000 population: 16.3</li>\n</ul>\n<p>Source: <a href="https://wonder.cdc.gov/controller/saved/D158/D321F156">National Vital Statistics System – Mortality Data (2021) via CDC WONDER </a></p>\n</div></div>']


In [24]:
list_sel = div_sel.xpath('.//li/text()')
list_sel

[<Selector query='.//li/text()' data='Number of alcoholic liver disease dea...'>,
 <Selector query='.//li/text()' data='Alcoholic liver disease deaths per 10...'>,
 <Selector query='.//li/text()' data='Number of alcohol-induced deaths, exc...'>,
 <Selector query='.//li/text()' data='Alcohol-induced deaths, excluding acc...'>]

In [25]:
list_sel.getall()

['Number of alcoholic liver disease deaths: 33,098',
 'Alcoholic liver disease deaths per 100,000 population: 10.0',
 'Number of alcohol-induced deaths, excluding accidents and homicides: 54,258',
 'Alcohol-induced deaths, excluding accidents and homicides per 100,000 population: 16.3']

In [26]:
list_sel.get()

'Number of alcoholic liver disease deaths: 33,098'